In [1]:
#Importar librerías
import pandas as pd
import os
import zipfile
import shutil
import requests
import zipfile
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import PercentFormatter
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap
from scipy.stats import iqr
import branca
import branca.colormap as cm
from branca.element import Template, MacroElement


In [2]:
#Definir directorio de trabajo
#Directorio de trabajo. Si existe directorio D:/, se selecciona si no, C:/
try:
    os.chdir('C:/users/claud/Documents/GitHub/mapas')
except:
    os.chdir('C:/users/claudio.pacheco/Documents/Github/mapas')


In [3]:
#crear directorio de salida
if not os.path.exists('datos'):
    os.makedirs('datos')

In [4]:
def download_and_extract(url, filename, check_file_path, extract_path):
    # Checar si el archivo ya existe
    if os.path.isfile(check_file_path):
        print(f"El archivo {filename} existe, no se descargará.")
    else:
        print(f"El archivo {filename} no existe. Descargando.")
        response = requests.get(url)
        with open(filename, "wb") as code:
            code.write(response.content)
        
        # extraer zip
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(extract_path)
        
        # Eliminar zip
        os.remove(filename)

In [5]:
descarga_manzanas=[{"url": "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/12_guerrero.zip",
                        "filename": "12_guerrero.zip",
                        "check_file_path": "datos/conjunto_de_datos/12m.shp",
                        "extract_path": "datos"}]
for file in descarga_manzanas:
    download_and_extract(file["url"], file["filename"], file["check_file_path"], file["extract_path"])

El archivo 12_guerrero.zip existe, no se descargará.


In [6]:
#Crear carpeta manzanas
if not os.path.exists("copernicus"):
    os.makedirs("copernicus")
download_and_extract("https://rapidmapping.emergency.copernicus.eu/backend/EMSR703/AOI01/GRA_MONIT01/EMSR703_AOI01_GRA_MONIT01_v1.zip?", "EMSR703_AOI01_GRA_MONIT01_v1", "copernicus/EMSR703_AOI01_GRA_MONIT01_builtUpA_v1.shp", "copernicus")
download_and_extract("https://rapidmapping.emergency.copernicus.eu/backend/EMSR703/AOI02/GRA_MONIT01/EMSR703_AOI02_GRA_MONIT01_v2.zip?", "EMSR703_AOI02_GRA_MONIT01_v2", "copernicus/EMSR703_AOI02_GRA_MONIT01_builtUpA_v2.shp", "copernicus")

El archivo EMSR703_AOI01_GRA_MONIT01_v1 existe, no se descargará.
El archivo EMSR703_AOI02_GRA_MONIT01_v2 existe, no se descargará.


In [7]:
if not os.path.exists("colonias"):
    os.makedirs("colonias")
download_and_extract("http://www.conapo.gob.mx/work/models/CONAPO/Marginacion/Datos_Abiertos/Colonia/imc2020_shp.zip", "imc2020_shp.zip", "colonias/colonias_imc2020.shp", "colonias")

El archivo imc2020_shp.zip existe, no se descargará.


In [8]:
#Leer shape de municipios
municipios=gpd.read_file("datos/conjunto_de_datos/12mun.shp")
#Filtrar municipios 12001, 12002, 12003, 12004, 12005, 12006, 12007, 12008, 12009, 12010, 12011
municipios=municipios[municipios["CVEGEO"].isin(["12001"])]
print(len(municipios))
municipios

1


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry
0,12001,12,001,Acapulco de Juárez,"MULTIPOLYGON (((2723457.629 539108.729, 272355..."


In [9]:
#leer shape de manzanas
manzanas=gpd.read_file("datos/conjunto_de_datos/12m.shp")
#Convertir crs a wgs84
manzanas = manzanas.to_crs(epsg=4326)
#Filtrar municipios 001, 003, 011, 014, 021, 030, 038, 048, 053, 054, 057
manzanas=manzanas[manzanas["CVE_MUN"].isin(["001"])]
manzanas.columns=[i.lower() for i in manzanas.columns]
#Si cve_mun es 001, cambiar nom_mun a Acapulco de Juárez, si es 021, cambiar a Coyuca de Benítez
manzanas.loc[manzanas["cve_mun"]=="001","nom_mun"]="Acapulco de Juárez"
manzanas

,cvegeo,cve_ent,cve_mun,cve_loc,cve_ageb,cve_mza,ambito,tipomza,geometry,nom_mun
0,1200100013912070,12,001,0001,3912,070,Urbana,Típica,"POLYGON ((-99.83317 16.91776, -99.83328 16.917...",Acapulco de Juárez
1,1200100010496015,12,001,0001,0496,015,Urbana,Típica,"POLYGON ((-99.91418 16.83146, -99.91413 16.831...",Acapulco de Juárez
2,1200100016832011,12,001,0001,6832,011,Urbana,Típica,"POLYGON ((-99.80206 16.84505, -99.80205 16.845...",Acapulco de Juárez
3,1200101731719011,12,001,0173,1719,011,Urbana,Típica,"POLYGON ((-99.73595 17.10427, -99.73596 17.104...",Acapulco de Juárez
4,1200100016419036,12,001,0001,6419,036,Urbana,Típica,"POLYGON ((-99.80175 16.82337, -99.80181 16.823...",Acapulco de Juárez
...,...,...,...,...,...,...,...,...,...,...
19662,1200100013132003,12,001,0001,3132,003,Urbana,Típica,"POLYGON ((-99.95349 16.89762, -99.95347 16.897...",Acapulco de Juárez
19663,1200100014906025,12,001,0001,4906,025,Urbana,Típica,"POLYGON ((-99.81672 16.81413, -99.81672 16.814...",Acapulco de Juárez
19664,1200100010477030,12,001,0001,0477,030,Urbana,Típica,"POLYGON ((-99.84340 16.85410, -99.84333 16.854...",Acapulco de Juárez
19665,120010001479A014,12,001,0001,479A,014,Urbana,Típica,"POLYGON ((-99.82445 16.81230, -99.82409 16.812...",Acapulco de Juárez


In [10]:
#Leer la capa de colonias
colonias=gpd.read_file("colonias/colonias_imc2020.shp")
#Convertir crs a wgs84
colonias = colonias.to_crs(epsg=4326)
colonias.columns=[i.lower() for i in colonias.columns]
#Filtrar Guerrero
colonias=colonias[colonias["cve_ent"]=="12"]
#Filtrar colonia de Acapulco de Juárez
colonias=colonias[colonias["mun"]=="001"]

#Dejar solo las columnas
colonias=colonias[['colonia','cp','gm_2020', "imn_2020",'geometry']]


In [11]:
#Pegar colonias a manzanas
manzanasc=gpd.sjoin(manzanas,colonias,how="left",op="intersects")
#Eliminar duplicados
manzanasc=manzanasc.drop_duplicates(subset=['cvegeo'])
print(len(manzanasc))
#Eliminar index_right
manzanasc=manzanasc.drop(columns=["index_right"])
manzanasc

c:\Users\claud\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


19667


,cvegeo,cve_ent,cve_mun,cve_loc,cve_ageb,cve_mza,ambito,tipomza,geometry,nom_mun,colonia,cp,gm_2020,imn_2020
0,1200100013912070,12,001,0001,3912,070,Urbana,Típica,"POLYGON ((-99.83317 16.91776, -99.83328 16.917...",Acapulco de Juárez,Las Torres,39700,Muy alto,0.911884
1,1200100010496015,12,001,0001,0496,015,Urbana,Típica,"POLYGON ((-99.91418 16.83146, -99.91413 16.831...",Acapulco de Juárez,Las Playas,39390,Bajo,0.964193
2,1200100016832011,12,001,0001,6832,011,Urbana,Típica,"POLYGON ((-99.80206 16.84505, -99.80205 16.845...",Acapulco de Juárez,Los Tulipanes,39905,Bajo,0.966033
3,1200101731719011,12,001,0173,1719,011,Urbana,Típica,"POLYGON ((-99.73595 17.10427, -99.73596 17.104...",Acapulco de Juárez,NaN,NaN,NaN,NaN
4,1200100016419036,12,001,0001,6419,036,Urbana,Típica,"POLYGON ((-99.80175 16.82337, -99.80181 16.823...",Acapulco de Juárez,La Marquesa,39906,Bajo,0.968724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19662,1200100013132003,12,001,0001,3132,003,Urbana,Típica,"POLYGON ((-99.95349 16.89762, -99.95347 16.897...",Acapulco de Juárez,Reforma Agraria,39406,Muy alto,0.912825
19663,1200100014906025,12,001,0001,4906,025,Urbana,Típica,"POLYGON ((-99.81672 16.81413, -99.81672 16.814...",Acapulco de Juárez,19 de Noviembre,39906,Muy alto,0.911864
19664,1200100010477030,12,001,0001,0477,030,Urbana,Típica,"POLYGON ((-99.84340 16.85410, -99.84333 16.854...",Acapulco de Juárez,Balcones de Costa Azul,39840,Medio,0.939635
19665,120010001479A014,12,001,0001,479A,014,Urbana,Típica,"POLYGON ((-99.82445 16.81230, -99.82409 16.812...",Acapulco de Juárez,Guadalupana,39907,Muy alto,0.906986


In [12]:
#Copernicus
copernicus1=gpd.read_file("copernicus/EMSR703_AOI01_GRA_MONIT01_builtUpA_v1.shp")
#Dejar solo columna simplified, det_method, damage_gra
copernicus1=copernicus1[["simplified","det_method","damage_gra","geometry"]]
#Traducir al español
copernicus1["simplified"]=copernicus1["simplified"].replace("Residential","Residencial")
copernicus1["simplified"]=copernicus1["simplified"].replace("Non-residential","No residencial")
copernicus1["det_method"]=copernicus1["det_method"].replace("Photo-interpretation","Foto-interpretación")
copernicus1["damage_gra"]=copernicus1["damage_gra"].replace("Destroyed","Destruido")
copernicus1["damage_gra"]=copernicus1["damage_gra"].replace("Damaged","Dañado")
copernicus1["damage_gra"]=copernicus1["damage_gra"].replace("Possibly damaged","Posiblemente dañado")
copernicus2=gpd.read_file("copernicus/EMSR703_AOI02_GRA_MONIT01_builtUpA_v2.shp")
#Dejar solo columna simplified, det_method, damage_gra
copernicus2=copernicus2[["simplified","det_method","damage_gra","geometry"]]
#Traducir al español
copernicus2["simplified"]=copernicus2["simplified"].replace("Residential","Residencial")
copernicus2["simplified"]=copernicus2["simplified"].replace("Non-residential","No residencial")
copernicus2["det_method"]=copernicus2["det_method"].replace("Photo-interpretation","Foto-interpretación")
copernicus2["damage_gra"]=copernicus2["damage_gra"].replace("Destroyed","Destruido")
copernicus2["damage_gra"]=copernicus2["damage_gra"].replace("Damaged","Dañado")
copernicus2["damage_gra"]=copernicus2["damage_gra"].replace("Possibly damaged","Posiblemente dañado")
#Unir copernicus1 y copernicus2
copernicus=pd.concat([copernicus1,copernicus2])
#Eliminar geometrías repetidas
copernicus=copernicus.drop_duplicates(subset="geometry")
#Eliminar geometrías vacías
copernicus=copernicus[copernicus["geometry"].notna()]
copernicus

,simplified,det_method,damage_gra,geometry
0,Residencial,Foto-interpretación,Destruido,"POLYGON ((-99.94346 16.90968, -99.94343 16.909..."
1,Residencial,Foto-interpretación,Dañado,"POLYGON ((-99.96490 16.91972, -99.96529 16.918..."
2,Residencial,Foto-interpretación,Dañado,"POLYGON ((-99.96613 16.92061, -99.96687 16.919..."
3,Residencial,Foto-interpretación,Dañado,"POLYGON ((-99.96746 16.92261, -99.96601 16.920..."
4,Residencial,Foto-interpretación,Dañado,"POLYGON ((-99.93357 16.86830, -99.93315 16.868..."
...,...,...,...,...
4272,Residencial,Foto-interpretación,Destruido,"POLYGON ((-99.85874 16.89650, -99.85879 16.896..."
4273,Residencial,Foto-interpretación,Destruido,"POLYGON ((-99.85925 16.88955, -99.85919 16.889..."
4274,Residencial,Foto-interpretación,Dañado,"POLYGON ((-99.85744 16.89958, -99.85753 16.899..."
4275,Residencial,Foto-interpretación,Dañado,"POLYGON ((-99.85099 16.87901, -99.85174 16.879..."


In [28]:
#Leer datos de espacio público
espacio=gpd.read_file("datos/conjunto_de_datos/12sip.shp")
espacio = espacio.to_crs(epsg=4326)
#Filtrar para 
espacio=espacio[espacio["CVE_MUN"].isin(["001"])]
espacio.columns=[i.lower() for i in espacio.columns]
print(espacio["tipo"].unique())
espacio=espacio[espacio["tipo"].isin(["Palacio de Gobierno", "Preescolar", "Parque", 
                                      "Secundaria", "Primaria", "Centro de Salud", "Medio Superior", 
                                      "Cancha", "Superior", "Hospital", "Centro de Abastos", 
                                      "Unidad Deportiva", "Museo", 
                                      "Palacio Municipal", "Áreas Verdes", 
                                      "Jardín"])]
espacio=gpd.sjoin(espacio,copernicus,how="left",op="intersects")
#Generar lat y lon
espacio["lat"]=espacio["geometry"].centroid.y
espacio["lon"]=espacio["geometry"].centroid.x
#Eliminar geometría
espacio=espacio.drop(columns=["geometry"])
#Eliminar duplicados
espacio=espacio.drop_duplicates(subset=["lat","lon"])
#Conservar los que no tengan nan en damage_gra
espacio=espacio[espacio["damage_gra"].notna()]
print(espacio["geografico"].unique())

['Palacio de Gobierno' 'Tanque Elevado' 'Preescolar' 'No Aplica' 'Parque'
 'Secundaria' 'Primaria' 'Centro de Salud'
 'Antena de Microondas de Telefonía' 'Medio Superior' 'Agua'
 'Central de Autobuses' 'Cancha' 'Balneario' 'Superior' 'Hospital'
 'Centro de Rehabilitación' 'Centro de Abastos' 'Centro de Investigación'
 'Centro de Asistencia Social' 'Gasolinera' 'Unidad Deportiva'
 'Campo de Golf' 'Museo' 'Plaza de Toros' 'Caja de Agua'
 'Monumento Histórico' 'Planta de Tratamiento de Agua' 'Palacio Municipal'
 'Áreas Verdes' 'Reclusorio' 'Jardín' 'Internacional']
['Escuela' 'Instalación Deportiva o Recreativa'
 'Instalación Gubernamental' 'Centro de Asistencia Médica'
 'Instalación Diversa']


c:\Users\claud\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\claud\AppData\Local\Temp\ipykernel_58844\772894458.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espacio["lat"]=espacio["geometry"].centroid.y
C:\Users\claud\AppData\Local\Temp\ipykernel_58844\772894458.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  espacio["lon"]=espacio["geometry"].centroid.x


In [14]:
espacio["geografico"].unique()

array(['Escuela', 'Instalación Deportiva o Recreativa',
       'Instalación Gubernamental', 'Centro de Asistencia Médica',
       'Instalación Diversa'], dtype=object)

In [15]:
#Dejar Palacio de Gobierno, Preescolar, Parque, Secundaria, Primaria, Centro de Salud, Medio Superior, Cancha, Superior, Hospital, Centro de Abastos, Unidad Deportiva, Museo, Palacio Municipal, Áreas Verdes, Jardín


In [16]:
#Hacer unión espacial entre manzanas y copernicus. El número de manzanas debe ser igual al número de manzanas en la base de manzanas
manzanas_copernicus=gpd.sjoin(manzanasc,copernicus,how="left",predicate="intersects")
#Eliminar duplicados
manzanas_copernicus=manzanas_copernicus.drop_duplicates(subset=['cvegeo'])
#Eliminar index_right
manzanas_copernicus=manzanas_copernicus.drop(columns=["index_right"])
#Pegar colonias a manzanas
manzanas_copernicus=gpd.sjoin(manzanas_copernicus,colonias,how="left",predicate='intersects')
#Eliminar duplicados
manzanas_copernicus=manzanas_copernicus.drop_duplicates(subset=['cvegeo'])
#Poner dummy de dañado o destruido
manzanas_copernicus["total_damage"]=np.where((manzanas_copernicus["damage_gra"]=="Destruido") | (manzanas_copernicus["damage_gra"]=="Dañado"),1,0)
#Sustituir nans de colonia_left, gm_2020_left, simplfied y damage_gra por No identificado

manzanas_copernicus["colonia_left"]=manzanas_copernicus["colonia_left"].fillna("No identificado")
manzanas_copernicus["gm_2020_left"]=manzanas_copernicus["gm_2020_left"].fillna("No identificado")
manzanas_copernicus["simplified"]=manzanas_copernicus["simplified"].fillna("No identificado")
manzanas_copernicus["damage_gra"]=manzanas_copernicus["damage_gra"].fillna("No identificado")
#Rellenar mza con 0
#Eliminar no identificados de damage_gra
manzanas_copernicus=manzanas_copernicus[manzanas_copernicus["damage_gra"]!="No identificado"]
manzanas_copernicus.columns
len(manzanas_copernicus)

15693

In [17]:
#Dejar solo cvegeo, total_damage, simplfied, damage_gra, geometry, colonia_left, gm_2020_left, tvivparhab, ocupvivpar, prom_ocup, mza y geometry
manzanas_copernicus=manzanas_copernicus[['cvegeo',"nom_mun",'total_damage','simplified','damage_gra','geometry','colonia_left','gm_2020_left', 'imn_2020_right']]

In [18]:
manzanas_copernicus["damage_gra"].unique()

array(['Dañado', 'Posiblemente dañado', 'Destruido'], dtype=object)

In [32]:
colormap_copernicus1 = {
    "Destruido": "#bd0026",  # bright red
    "Dañado": "#fd8d3c",    # deep purple
    "Posiblemente dañado": "#ffffb2"  # gold/yellow for contrast
}

def get_color(feature):
    gm_value = feature['properties']['damage_gra']
    return colormap_copernicus1.get(gm_value, '#FFFFFF')

geografico_color_map = {
    "Instalación Gubernamental": "#00008B",
    "Escuela": "#FFFF00",
    "Instalación Deportiva o Recreativa": "#c51b8a",
    "Centro de Asistencia Médica": "#33a02c",
    "Instalación Diversa": "#b15928"
}

# Function to get color based on 'geografico' value
def get_geografico_color(geografico_value):
    return geografico_color_map.get(geografico_value, "#FFFFFF")  # Default to white if not in map


mapa = folium.Map(location=[16.85713, -99.88510], zoom_start=13, control_scale=True, tiles=None)

# Add the Google Satellite tile layer as the only layer
tile_layer = folium.TileLayer(
    tiles='http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}',
    attr="@claudiodanielpc con información de INEGI y Copernicus",
    name='Mapa base: Google Satellite',
    overlay=False,
    control=True
)
tile_layer.add_to(mapa)

#Añadir shape de municipios. Fondo transparente, contorno naranja. Popup con nombre de municipio
municipio_marker=folium.FeatureGroup(name="División municipal")
folium.GeoJson(municipios,style_function=lambda x: {"fillColor":"#ffffff","color":"#ff9900","weight":3,"fillOpacity":0.1},
               highlight_function=lambda x: {"fillColor":"#000000","color":"#000000","weight":1,"fillOpacity":0.1},
               tooltip=folium.GeoJsonTooltip(fields=["NOMGEO"],aliases=["Municipio: "])).add_to(municipio_marker)
municipio_marker.add_to(mapa)


colonia_marker=folium.FeatureGroup(name="Colonias",show=True)
#Agregar capa de colonias
folium.GeoJson(
    colonias,
    style_function=lambda feature: {
        #Rellenar por grado de marginación
        'fillColor': "Transparent",

        'fillOpacity': 0.5,
        'color': '#000000',   # You can adjust the border color if needed
        'weight': 1,
        'dashArray': '5, 5'  # Dashed borders, remove this if not desired
    },
tooltip=folium.GeoJsonTooltip(fields=["colonia"],aliases=["Colonia: "])).add_to(colonia_marker
                                                                                                    )
colonia_marker.add_to(mapa)



#Agregar capa de copernicus1
manzanasgroup = folium.FeatureGroup(name="Mazanas",show=True)

folium.GeoJson(
    manzanas_copernicus,
    style_function=lambda feature: {
        #Rellenar por grado de marginación
        'fillColor': get_color(feature),
        
        'fillOpacity': 0.5,
        'color': '#000000',   # You can adjust the border color if needed
        'weight': 1,
        'dashArray': '5, 5'  # Dashed borders, remove this if not desired
    },
tooltip=folium.GeoJsonTooltip(fields=["damage_gra","nom_mun","colonia_left", "gm_2020_left"],aliases=["Tipo de daño","Municipio: ","Colonia: ",
                                                                                               
                                                                                      "Grado de marginación: "])).add_to(manzanasgroup)
manzanasgroup.add_to(mapa)


#Incorporar capa de espacio público
espacio_marker=folium.FeatureGroup(name="Espacio público",show=True)
#Agregar capa de espacio público con circle marker
for i in range(len(espacio)):
    geografico_value = espacio.iloc[i]["geografico"]
    color = get_geografico_color(geografico_value)

    folium.CircleMarker(location=[espacio.iloc[i]["lat"],espacio.iloc[i]["lon"]],
                        radius=5,
                        color=color,
                        fill=True,
                        fill_color=color,
                        fill_opacity=0.5,
                        tooltip=folium.Tooltip(text=f"<b>Categoria:</b> {espacio.iloc[i]['geografico']}<br><b>Tipo:</b> {espacio.iloc[i]['tipo']}<br><b>Nombre:</b> {espacio.iloc[i]['nomserv']}")).add_to(espacio_marker)
espacio_marker.add_to(mapa)

# Define the HTML template for the geografico legend
geografico_legend_template = """
{% macro html(this, kwargs) %}
<div style="position: fixed; 
     top: 50px; left: 50px; width: 300px; height: 150px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; opacity: 0.8; padding: 5px;">
    &nbsp; <b>Categorías Espacios</b> <br>
    &nbsp; Instalación Gubernamental &nbsp; <i style="background: #00008B; width: 20px; height: 10px; display: inline-block;"></i><br>
    &nbsp; Escuela &nbsp; <i style="background: #FFFF00; width: 20px; height: 10px; display: inline-block;"></i><br>
    &nbsp; Instalación Deportiva o Recreativa &nbsp; <i style="background: #c51b8a; width: 20px; height: 10px; display: inline-block;"></i><br>
    &nbsp; Centro de Asistencia Médica &nbsp; <i style="background: #33a02c; width: 20px; height: 10px; display: inline-block;"></i><br>
    &nbsp; Instalación Diversa &nbsp; <i style="background: #808080; width: 20px; height: 10px; display: inline-block;"></i><br>
</div>
{% endmacro %}
"""

# Create a MacroElement for the new legend
geografico_macro = MacroElement()
geografico_macro._template = Template(geografico_legend_template)

# Add the new legend to the map
mapa.get_root().add_child(geografico_macro)


# Define the HTML template for the legend
template = """
{% macro html(this, kwargs) %}
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 220px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; opacity: 0.8; padding: 5px;">
    &nbsp; <b>Tipo de daño</b> <br>
    &nbsp; Posiblemente dañado &nbsp; <i style="background: #ffffb2; width: 20px; height: 10px; display: inline-block;"></i><br>
    &nbsp; Dañado &nbsp; <i style="background: #fd8d3c; width: 20px; height: 10px; display: inline-block;"></i><br>
    &nbsp; Destruido &nbsp; <i style="background: #bd0026; width: 20px; height: 10px; display: inline-block;"></i><br>
</div>
{% endmacro %}
"""



# The URL to your CSV or Excel file hosted on GitHub or another service
file_url = 'https://github.com/labdatos-se/guerrero_vivi/raw/main/viviendas_acapulco.xlsx'


html = f"""
<div style="position: fixed; bottom: 50px; left: 80%; z-index: 9999; background-color: white; padding: 10px; 
     border: 1px solid black; opacity: 0.8; font-size: 12px;">
    <a href="{file_url}" target="_blank" style="text-decoration: none; color: black;">Descargar la tabla de manzanas y viviendas</a>
</div>
"""

# Create a new map with the custom HTML
mapa.get_root().html.add_child(folium.Element(html))


# Create a MacroElement which will be added to the map
macro = MacroElement()
macro._template = Template(template)

# Add the legend to the map
mapa.get_root().add_child(macro)
folium.LayerControl().add_to(mapa)

# To display the map in a Jupyter notebook, you can simply use
mapa

TypeError: string indices must be integers

In [23]:
#Guardar mapa
mapa.save("espacios_acapulco.html")

In [24]:
# Read in the saved map HTML
with open("espacios_acapulco.html", 'r') as file:
    html = file.read()

# Add a title tag; replace 'Your Map Title' with your desired title
title_tag = "<title>Espacios susceptibles de afectación en Acapulco por Otis</title>"
head_tag = "<head>"
# Insert the title tag after the head tag
html = html.replace(head_tag, head_tag + title_tag, 1)

# Write the modified HTML back to the file
with open("espacios_acapulco.html", 'w') as file:
    file.write(html)